# Variant calling & filtering with AvePla samples

In [ ]:
module load bcftools

## Calling all invariant+variant sites 
*bcftools* mpileup ... | *bcftools* call ...

In [ ]:
baseDIR=~/snap_hap_repHZ/SnpCalls
bamlist=$baseDIR/AvePla.bams.list
outVCF=$baseDIR/AvePla.vcf.gz
sbatch -J SnpCallingJob ./SnpCalling.sbatch.sh $baseDIR $bamlist $outVCF
11:16:30

In [ ]:
bcftools query -f '%CHROM\t%POS\t%QUAL\t%INFO/DP\t%INFO/MQ\t%INFO/AN\t%INFO/AC\n' AvePla.vcf.gz > AvePla.vcf.info

In [ ]:
## Filter
baseDIR=~/snap_hap_repHZ/SnpCalls

# Step 1: Remove SNPs within 5bp of INDELs and keep only bi-alleleic variant sites
time bcftools filter --threads 4 $inVCF --SnpGap 5 | \
     bcftools view --threads 4 - -Oz -o $outVCF -m2 -M2 -v snps -e "AC==0 || AC==AN" --write-index
inVCF=$baseDIR/AvePla.vcf.gz
outVCF=$baseDIR/AvePla.biSNPs.vcf.gz
sbatch -J SnpFilteringJob1 ./SnpFiltering.sbatch.sh $inVCF $outVCF

#263m40.339s

# Step 2: Remove sites based on depth, mapping quality, and QUAL
inVCF=$baseDIR/AvePla.biSNPs.vcf.gz
outVCF=$baseDIR/AvePla.biSNPs.filtered.vcf.gz
sbatch -J Filter ./SnpFiltering.sbatch.sh $inVCF $outVCF
# 12 mins

# Step 3: Remove sites that have a missing fraction > 0.8
inVCF=$baseDIR/AvePla.biSNPs.filtered.vcf.gz
outVCF=$baseDIR/AvePla.biSNPs.filtered.missLT80.vcf.gz
sbatch -J Filter3 ./SnpFiltering.sbatch.sh $inVCF $outVCF
# 20 mins

In [ ]:
bcftools query -f '%CHROM\t%POS\t%QUAL\t%INFO/DP\t%INFO/MQ\t%INFO/AN\t%INFO/AC\n' ./AvePla.biSNPs.filtered.bcf > AvePla.biSNPs.filtered.vcf.info

In [ ]:
## Run STITCH


In [ ]:
# STITCH variables
K=75
downsampleToCov=20
bx_tag=TRUE
ngen=100
niter=40
expRate=0.5
plot=FALSE
bamlist=~/snap_hap/sample_info/bam_info/bams_Am_all.txt
outputDIR=$stitchDIR/$chrom/stitch_chromSegments/$(basename ${posfile/.pos})_K${K}_cov${downsampleToCov}_bxTRUE_niter${niter}_ngen${ngen}_r${expRate}_plotFALSE

In [ ]:
No. of segments for each chromosome:

Chr1 71919034: 72 segments
Chr2 77118269: 78 segments
Chr3 65231163: 66 segments
Chr4 54887108: 55 segments
Chr5 71106538: 72 segments
Chr6 55699338: 56 segments
Chr7 55564713: 56 segments
Chr8 57431585: 58 segments

In [3]:
wc -l ~/snap_hap_repHZ/Stitch/Chr?/*pos

  1610316 /nfs/scistore18/bartogrp/apal/snap_hap_repHZ/Stitch/Chr1/Chr1.AvePla.biSNPs.filtered.missLT80.pos
  1790609 /nfs/scistore18/bartogrp/apal/snap_hap_repHZ/Stitch/Chr2/Chr2.AvePla.biSNPs.filtered.missLT80.pos
  1424535 /nfs/scistore18/bartogrp/apal/snap_hap_repHZ/Stitch/Chr3/Chr3.AvePla.biSNPs.filtered.missLT80.pos
  1186579 /nfs/scistore18/bartogrp/apal/snap_hap_repHZ/Stitch/Chr4/Chr4.AvePla.biSNPs.filtered.missLT80.pos
  1728443 /nfs/scistore18/bartogrp/apal/snap_hap_repHZ/Stitch/Chr5/Chr5.AvePla.biSNPs.filtered.missLT80.pos
  1254284 /nfs/scistore18/bartogrp/apal/snap_hap_repHZ/Stitch/Chr6/Chr6.AvePla.biSNPs.filtered.missLT80.pos
  1195630 /nfs/scistore18/bartogrp/apal/snap_hap_repHZ/Stitch/Chr7/Chr7.AvePla.biSNPs.filtered.missLT80.pos
  1384030 /nfs/scistore18/bartogrp/apal/snap_hap_repHZ/Stitch/Chr8/Chr8.AvePla.biSNPs.filtered.missLT80.pos
 11574426 total


In [ ]:
chrom=Chr6
K=75
downsampleToCov=10
bx_tag=TRUE
ngen=100
niter=40
expRate=0.5
plot=FALSE
bamlist=~/snap_hap_repHZ/SnpCalls/AvePla.bams.list
posfile=~/snap_hap_repHZ/Stitch/Chr6/Chr6.AvePla.biSNPs.filtered.missLT80.pos
outputDIR=~/snap_hap_repHZ/Stitch/Chr6

chromStart=52000000
chromEnd=53000000
buffer=10000

STITCH.R --chr=$chrom \
    --regionStart=$chromStart \
    --regionEnd=$chromEnd \
    --buffer=$buffer \
    --K=$K \
    --downsampleToCov=$downsampleToCov \
    --use_bx_tag=TRUE \
    --niterations=$niter \
    --expRate=$expRate  \
    --nGen=$ngen \
    --plotAfterImputation=FALSE \
    --plotHapSumDuringIterations=FALSE \
    --plot_shuffle_haplotype_attempts=FALSE \
    --bamlist=$bamlist \
    --posfile=$posfile \
    --outputdir=$outputDIR \
    --nCores=10


--regionStart=$chromStart \
--regionEnd=$chromEnd \
--buffer=$buffer \

In [ ]:
baseDIR=~/snap_hap_repHZ/Stitch/
buffer=100000
K=75
cov=10 
use_bx_tag=TRUE
ngen=100
niter=40 
expRate=0.5 
plot=FALSE

chrom=Chr1 ## ALL DONE
cd $baseDIR/$chrom
sbatch --array=2-72 -J imp1 ~/snap_hap_repHZ/Stitch/_scripts/Stitch.sbatch.sh $chrom $buffer $K $cov ${use_bx_tag} $ngen $niter $expRate $plot

chrom=Chr2
cd $baseDIR/$chrom
sbatch --array=72-75,2-3 -J imp2 ~/snap_hap_repHZ/Stitch/_scripts/Stitch.sbatch.sh $chrom $buffer $K $cov ${use_bx_tag} $ngen $niter $expRate $plot
# sbatch --array=1-78 -J imp2 ~/snap_hap_repHZ/Stitch/_scripts/Stitch.sbatch.sh $chrom $buffer $K $cov ${use_bx_tag} $ngen $niter $expRate $plot

chrom=Chr3
cd $baseDIR/$chrom
sbatch --array=2,62-65 -J imp3 ~/snap_hap_repHZ/Stitch/_scripts/Stitch.sbatch.sh $chrom $buffer $K $cov ${use_bx_tag} $ngen $niter $expRate $plot
# sbatch --array=1-66 -J imp3 ~/snap_hap_repHZ/Stitch/_scripts/Stitch.sbatch.sh $chrom $buffer $K $cov ${use_bx_tag} $ngen $niter $expRate $plot

chrom=Chr4
cd $baseDIR/$chrom
sbatch --array=1-55 -J imp4 ~/snap_hap_repHZ/Stitch/_scripts/Stitch.sbatch.sh $chrom $buffer $K $cov ${use_bx_tag} $ngen $niter $expRate $plot

chrom=Chr5
cd $baseDIR/$chrom
sbatch --array=2-4,70-72 -J imp5 ~/snap_hap_repHZ/Stitch/_scripts/Stitch.sbatch.sh $chrom $buffer $K $cov ${use_bx_tag} $ngen $niter $expRate $plot
# sbatch --array=1-72 -J imp5 ~/snap_hap_repHZ/Stitch/_scripts/Stitch.sbatch.sh $chrom $buffer $K $cov ${use_bx_tag} $ngen $niter $expRate $plot

chrom=Chr6
cd $baseDIR/$chrom
sbatch --array=1-56 -J imp6 ~/snap_hap_repHZ/Stitch/_scripts/Stitch.sbatch.sh $chrom $buffer $K $cov ${use_bx_tag} $ngen $niter $expRate $plot

chrom=Chr7
cd $baseDIR/$chrom
sbatch --array=55,52 -J imp7 ~/snap_hap_repHZ/Stitch/_scripts/Stitch.sbatch.sh $chrom $buffer $K $cov ${use_bx_tag} $ngen $niter $expRate $plot
# sbatch --array=1-56 -J imp7 ~/snap_hap_repHZ/Stitch/_scripts/Stitch.sbatch.sh $chrom $buffer $K $cov ${use_bx_tag} $ngen $niter $expRate $plot

chrom=Chr8
cd $baseDIR/$chrom
sbatch --array=54-57 -J imp8 ~/snap_hap_repHZ/Stitch/_scripts/Stitch.sbatch.sh $chrom $buffer $K $cov ${use_bx_tag} $ngen $niter $expRate $plot
# sbatch --array=1-58 -J imp8 ~/snap_hap_repHZ/Stitch/_scripts/Stitch.sbatch.sh $chrom $buffer $K $cov ${use_bx_tag} $ngen $niter $expRate $plot

In [ ]:
# Postprocessing
## Change PATH
export PATH=~/.local/bin:$PATH

baseDIR=~/snap_hap_repHZ/Stitch
chrom=Chr6

stitchVcfList=$baseDIR/$chrom/stitchVCFs.highConf.list
outVCF=$baseDIR/$chrom/$chrom.AvePla.stitch.vcf.gz
realpath $baseDIR/$chrom/stitch_chromSegments/*/stitch.*.vcf.gz  > $stitchVcfList

for vcf in `cat $stitchVcfList`; do echo $vcf; bcftools tabix $vcf; done
bcftools concat --allow-overlaps --threads 4 -Oz -o $outVCF -f $stitchVcfList

time bash ~/snap_hap/_scripts/bash/stitch/add_PG_PL.sh $outVCF

time bash ~/snap_hap/_scripts/bash/fill-AC-AN_vcfgzFormat.sh $outVCF
time bash ~/snap_hap/_scripts/bash/fill-AC-AN_vcfgzFormat.sh ${outVCF/.vcf.gz/.PL.vcf.gz}

## Keep only variant sites
time bcftools view -e 'AC==0 | AC==AN' -Oz -o ${outVCF/.vcf.gz/.SnpOnly.final.vcf.gz} ${outVCF/.vcf.gz/.PL.tagged.vcf.gz}

## Extract STITCH parameters
time bcftools query -f "%CHROM\t%POS\t%EAF\t%INFO_SCORE\t%HWE\t%ERC\t%EAC\t%PAF\t%AN\t%AC" ${outVCF/.vcf.gz/.SnpOnly.final.vcf.gz} > ${outVCF/.vcf.gz/.SnpOnly.final.params.info}

# ## Calculate missing genotypes per site and individual
# echo -e '\nCalculating fraction of missing data'
# time vcftools --gzvcf $outVCF --missing-indv --out ${chrom}_missing-indv
# time vcftools --gzvcf $outVCF --missing-site --out ${chrom}_missing-site

# ## Calculate heterozygosity per individual
# echo -e '\nCalculating heterozygosity'
# time vcftools --gzvcf $outVCF --het --out ${chrom}_het-indv

In [ ]:
# Concatenate all Stitch VCFs
baseDIR=~/snap_hap_repHZ
cd $baseDIR

bcftools concat -a --threads 8 $baseDIR/Stitch/Chr?/Chr?.AvePla.stitch.SnpOnly.final.vcf.gz -Oz -o $baseDIR/Stitch/AvePla.stitch.SnpOnly.final.vcf.gz
bcftools tabix $baseDIR/Stitch/AvePla.stitch.SnpOnly.final.vcf.gz

In [ ]:
bcftools view -S $baseDIR/samples/samples.AvePla.sorted.txt ./AvePla.stitch.SnpOnly.final.vcf.gz -Oz -o ./AvePla.stitch.SnpOnly.final.sorted.vcf.gz
bcftools tabix $baseDIR/Stitch/AvePla.stitch.SnpOnly.final.sorted.vcf.gz
